In [1]:
using QuantumOptics
using Einsum

┌ Warning: Error requiring `KrylovKit` from `LinearSolve`
│   exception =
│    LoadError: ArgumentError: Package LinearSolve does not have KrylovKit in its dependencies:
│    - You may have a partially installed environment. Try `Pkg.instantiate()`
│      to ensure all packages in the environment are installed.
│    - Or, if you have LinearSolve checked out for development and have
│      added KrylovKit as a dependency but haven't updated your primary
│      environment's manifest file, try `Pkg.resolve()`.
│    - Otherwise you may need to report an issue with LinearSolve
│    Stacktrace:
│      [1] macro expansion
│        @ ./loading.jl:1167 [inlined]
│      [2] macro expansion
│        @ ./lock.jl:223 [inlined]
│      [3] require(into::Module, mod::Symbol)
│        @ Base ./loading.jl:1144
│      [4] include(mod::Module, _path::String)
│        @ Base ./Base.jl:419
│      [5] include(x::String)
│        @ LinearSolve ~/.julia/packages/LinearSolve/qCLK7/src/LinearSolve.jl:1
│      [

In [2]:
basis = NLevelBasis(2) 

NLevel(N=2)

In [3]:
t = -1

H1 = t * ( transition(basis,1,2) + transition(basis,2,1) )

Operator(dim=2x2)
  basis: NLevel(N=2)sparse([2, 1], [1, 2], ComplexF64[-1.0 - 0.0im, -1.0 - 0.0im], 2, 2)

In [4]:
E1, U1 = eigenstates(dense(H1))

([-1.0, 1.0], Ket{NLevelBasis{Int64}, Vector{ComplexF64}}[Ket(dim=2)
  basis: NLevel(N=2)
 -0.7071067811865475 + 0.0im
 -0.7071067811865475 + 0.0im, Ket(dim=2)
  basis: NLevel(N=2)
 -0.7071067811865475 + 0.0im
  0.7071067811865475 + 0.0im])

In [5]:
# projection
substates = 1:2
basis_tilde = SubspaceBasis(basis, U1[substates])

Subspace(superbasis=NLevel(N=2), states:2)

In [6]:
projector(basis_tilde, basis)

Operator(dim=2x2)
  basis left:  Subspace(superbasis=NLevel(N=2), states:2)
  basis right: NLevel(N=2)
 -0.707107-0.0im  -0.707107-0.0im
 -0.707107-0.0im   0.707107-0.0im

In [7]:
projector(basis, basis_tilde)

Operator(dim=2x2)
  basis left:  NLevel(N=2)
  basis right: Subspace(superbasis=NLevel(N=2), states:2)
 -0.707107+0.0im  -0.707107+0.0im
 -0.707107+0.0im   0.707107+0.0im

In [8]:
P1 = projector(basis, basis_tilde)
P1_dag = dagger(P1)

Operator(dim=2x2)
  basis left:  Subspace(superbasis=NLevel(N=2), states:2)
  basis right: NLevel(N=2)
 -0.707107-0.0im  -0.707107-0.0im
 -0.707107-0.0im   0.707107-0.0im

In [9]:
ψ = 3//5*nlevelstate(basis,1) + 4//5*nlevelstate(basis,2)

Ket(dim=2)
  basis: NLevel(N=2)
 0.6 + 0.0im
 0.8 + 0.0im

In [10]:
ψ = Ket(basis, [3//5,4//5])

Ket(dim=2)
  basis: NLevel(N=2)
 0.6
 0.8

In [11]:
P1_dag*ψ

Ket(dim=2)
  basis: Subspace(superbasis=NLevel(N=2), states:2)
 -0.9899494936611664 + 0.0im
 0.14142135623730956 + 0.0im

In [12]:
H1_sub = P1_dag*H1*P1

Operator(dim=2x2)
  basis: Subspace(superbasis=NLevel(N=2), states:2)
 -1.0+0.0im  0.0+0.0im
  0.0+0.0im  1.0+0.0im

In [13]:
# many-body basis
PN = 2 
states_mb = bosonstates(basis, PN)

3-element QuantumOpticsBase.SortedVector{Vector{Int64}, Base.Order.ReverseOrdering{Base.Order.ForwardOrdering}}:
 [2, 0]
 [1, 1]
 [0, 2]

In [14]:
basis_mb = ManyBodyBasis(basis, states_mb)

ManyBody(onebodybasis=NLevel(N=2), states:3)

In [15]:
H1_mb = manybodyoperator(basis_mb, H1)

Operator(dim=3x3)
  basis: ManyBody(onebodybasis=NLevel(N=2), states:3)sparse([2, 1, 3, 2], [1, 2, 2, 3], ComplexF64[-1.4142135623730951 - 0.0im, -1.4142135623730951 - 0.0im, -1.4142135623730951 - 0.0im, -1.4142135623730951 - 0.0im], 3, 3)

In [16]:
# Hubbard interaction
U0 = 0
n1 = number(basis_mb, 1)
n2 = number(basis_mb, 2)
Id = identityoperator(basis_mb)
Vint_mb = U0/2*(n1*(n1-Id) + n2*(n2-Id))

Operator(dim=3x3)
  basis: ManyBody(onebodybasis=NLevel(N=2), states:3)sparse(Int64[], Int64[], ComplexF64[], 3, 3)

In [17]:
H_mb = H1_mb + Vint_mb

Operator(dim=3x3)
  basis: ManyBody(onebodybasis=NLevel(N=2), states:3)sparse([2, 1, 3, 2], [1, 2, 2, 3], ComplexF64[-1.4142135623730951 + 0.0im, -1.4142135623730951 + 0.0im, -1.4142135623730951 + 0.0im, -1.4142135623730951 + 0.0im], 3, 3)

In [18]:
dense( H_mb )

Operator(dim=3x3)
  basis: ManyBody(onebodybasis=NLevel(N=2), states:3)
      0.0+0.0im  -1.41421+0.0im       0.0+0.0im
 -1.41421+0.0im       0.0+0.0im  -1.41421+0.0im
      0.0+0.0im  -1.41421+0.0im       0.0+0.0im

In [19]:
eigenenergies( dense(H_mb) )

3-element Vector{Float64}:
 -2.0
 -1.9949319973733282e-16
  2.000000000000001

In [20]:
EE, UU = eigenstates( H_mb, 3)

[ Info: Defaulting to sparse diagonalization for sparse operator. If storing the full operator is possible, it might be faster to do `eigenstates(dense(op))`. Set `info=false` to turn off this message.


([-1.9999999999999991, 2.6645352591003757e-15, 2.0000000000000004], Ket{ManyBodyBasis{NLevelBasis{Int64}, QuantumOpticsBase.SortedVector{Vector{Int64}, Base.Order.ReverseOrdering{Base.Order.ForwardOrdering}}, UInt64}, Vector{ComplexF64}}[Ket(dim=3)
  basis: ManyBody(onebodybasis=NLevel(N=2), states:3)
 0.4693716741659837 + 0.17230853574510274im
 0.6637917873992998 + 0.2436810681633735im
 0.4693716741659841 + 0.17230853574510263im, Ket(dim=3)
  basis: ManyBody(onebodybasis=NLevel(N=2), states:3)
 -0.6404482189056551 + 0.2997099913259059im
            -2.8e-16 - 2.2e-16im
  0.6404482189056546 - 0.29970999132590576im, Ket(dim=3)
  basis: ManyBody(onebodybasis=NLevel(N=2), states:3)
 -0.0223415745544634 + 0.4995006046507125im
 0.03159575773969178 - 0.7064005295105991im
 -0.0223415745544635 + 0.4995006046507125im])

In [21]:
# alternatively construct Vint from the 2-body matrix elements
# transition(basis,i,k)⊗transition(basis,j,l) goes with V_ijkl 

In [22]:
V2  = U0/2*transition(basis,1,1)⊗transition(basis,1,1) 
V2 += U0/2*transition(basis,2,2)⊗transition(basis,2,2) 

Operator(dim=4x4)
  basis: [NLevel(N=2) ⊗ NLevel(N=2)]sparse(Int64[], Int64[], ComplexF64[], 4, 4)

In [23]:
V2.data

4×4 SparseArrays.SparseMatrixCSC{ComplexF64, Int64} with 0 stored entries:
     ⋅          ⋅          ⋅          ⋅    
     ⋅          ⋅          ⋅          ⋅    
     ⋅          ⋅          ⋅          ⋅    
     ⋅          ⋅          ⋅          ⋅    

In [24]:
V2_mb = manybodyoperator(basis_mb, V2)

Operator(dim=3x3)
  basis: ManyBody(onebodybasis=NLevel(N=2), states:3)sparse(Int64[], Int64[], ComplexF64[], 3, 3)

In [25]:
isapprox(V2_mb, Vint_mb)

true

In [26]:
# compare the following for indexing the 2-ptl states
transition(basis,1,2)⊗transition(basis,1,1), transition(basis,1,1)⊗transition(basis,1,2)

(Operator(dim=4x4)
  basis: [NLevel(N=2) ⊗ NLevel(N=2)]sparse([1], [2], ComplexF64[1.0 + 0.0im], 4, 4), Operator(dim=4x4)
  basis: [NLevel(N=2) ⊗ NLevel(N=2)]sparse([1], [3], ComplexF64[1.0 + 0.0im], 4, 4))

In [27]:
P1M = P1.data
P1_dagM = P1_dag.data

2×2 adjoint(::Matrix{ComplexF64}) with eltype ComplexF64:
 -0.707107-0.0im  -0.707107-0.0im
 -0.707107-0.0im   0.707107-0.0im

In [28]:
H1M = H1.data

2×2 SparseArrays.SparseMatrixCSC{ComplexF64, Int64} with 2 stored entries:
      ⋅      -1.0-0.0im
 -1.0-0.0im       ⋅    

In [29]:
V2M = V2.data

4×4 SparseArrays.SparseMatrixCSC{ComplexF64, Int64} with 0 stored entries:
     ⋅          ⋅          ⋅          ⋅    
     ⋅          ⋅          ⋅          ⋅    
     ⋅          ⋅          ⋅          ⋅    
     ⋅          ⋅          ⋅          ⋅    

In [30]:
P2M = (P1⊗P1).data
P2_dagM = (P1_dag⊗P1_dag).data

4×4 Matrix{ComplexF64}:
 0.5+0.0im   0.5+0.0im   0.5+0.0im   0.5+0.0im
 0.5+0.0im  -0.5+0.0im   0.5+0.0im  -0.5+0.0im
 0.5+0.0im   0.5+0.0im  -0.5+0.0im  -0.5+0.0im
 0.5+0.0im  -0.5+0.0im  -0.5+0.0im   0.5-0.0im

In [31]:
@einsum H1_tildeM[i,j] := P1_dagM[i,k] * H1M[k,l] * P1M[l,j];
@einsum V2_tildeM[i2,j2] :=  P2_dagM[i2,k2] * V2M[k2,l2] * P2M[l2,j2];
@einsum U4[k,l,m,n] := P1M[k,i] * P1M[l,i] * P1_dagM[i,m] * P1_dagM[i,n];

In [32]:
H1_tilde = Operator(basis_tilde, H1_tildeM)

Operator(dim=2x2)
  basis: Subspace(superbasis=NLevel(N=2), states:2)
 -1.0+0.0im  0.0+0.0im
  0.0+0.0im  1.0+0.0im

In [33]:
basis_mb_tilde  = ManyBodyBasis(basis_tilde, states_mb)

ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=2), states:2), states:3)

In [34]:
H_NI_tilde = manybodyoperator(basis_mb_tilde, H1_tilde)

Operator(dim=3x3)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=2), states:2), states:3)
 -2.0+0.0im  0.0+0.0im  0.0+0.0im
  0.0+0.0im  0.0+0.0im  0.0+0.0im
  0.0+0.0im  0.0+0.0im  2.0+0.0im

In [35]:
basis_tilde2 = basis_tilde⊗basis_tilde

[Subspace(superbasis=NLevel(N=2), states:2) ⊗ Subspace(superbasis=NLevel(N=2), states:2)]

In [36]:
Vint_tilde = Operator(basis_tilde2, U0/2*reshape(U4,4,4))

Operator(dim=4x4)
  basis: [Subspace(superbasis=NLevel(N=2), states:2) ⊗ Subspace(superbasis=NLevel(N=2), states:2)]
 0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im

In [37]:
V2_tilde = Operator(basis_tilde2, V2_tildeM)

Operator(dim=4x4)
  basis: [Subspace(superbasis=NLevel(N=2), states:2) ⊗ Subspace(superbasis=NLevel(N=2), states:2)]
 0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im

In [38]:
isapprox(Vint_tilde, V2_tilde)

true

In [39]:
Vint_tilde_mb = manybodyoperator(basis_mb_tilde, Vint_tilde)

Operator(dim=3x3)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=2), states:2), states:3)
 0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im

In [40]:
Vint_tilde_mb = manybodyoperator(basis_mb_tilde, V2_tilde)

Operator(dim=3x3)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=2), states:2), states:3)
 0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im

In [41]:
H_mb_tilde = H_NI_tilde + Vint_tilde_mb

Operator(dim=3x3)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=2), states:2), states:3)
 -2.0+0.0im  0.0+0.0im  0.0+0.0im
  0.0+0.0im  0.0+0.0im  0.0+0.0im
  0.0+0.0im  0.0+0.0im  2.0+0.0im

In [42]:
eigenenergies( dense(H_mb_tilde) )

3-element Vector{Float64}:
 -1.9999999999999996
  0.0
  1.9999999999999996

In [43]:
EEt, UUt = eigenstates(  H_mb_tilde )

([-1.9999999999999996, 0.0, 1.9999999999999996], Ket{ManyBodyBasis{SubspaceBasis{Vector{Int64}, NLevelBasis{Int64}, Ket{NLevelBasis{Int64}, Vector{ComplexF64}}, 0x868ccc38620d16b5, UInt64}, QuantumOpticsBase.SortedVector{Vector{Int64}, Base.Order.ReverseOrdering{Base.Order.ForwardOrdering}}, UInt64}, Vector{ComplexF64}}[Ket(dim=3)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=2), states:2), states:3)
 1.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im, Ket(dim=3)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=2), states:2), states:3)
 0.0 + 0.0im
 1.0 + 0.0im
 0.0 + 0.0im, Ket(dim=3)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=2), states:2), states:3)
 0.0 + 0.0im
 0.0 + 0.0im
 1.0 + 0.0im])

In [44]:
states_mb

3-element QuantumOpticsBase.SortedVector{Vector{Int64}, Base.Order.ReverseOrdering{Base.Order.ForwardOrdering}}:
 [2, 0]
 [1, 1]
 [0, 2]

In [45]:
state_list = []
for state in states_mb
    #println(findall(state.!=0))
    repr = Int64[]
    for oi in findall(state.!=0)
            push!(repr,repeat([oi],state[oi])...)
    end
    #println(repr)
    push!(state_list, repr)
end
state_list

3-element Vector{Any}:
 [1, 1]
 [1, 2]
 [2, 2]

In [46]:
state_index = Dict()
for (ind, state) in enumerate(state_list)
    state_index[state] = ind
end
state_index

Dict{Any, Any} with 3 entries:
  [1, 1] => 1
  [2, 2] => 3
  [1, 2] => 2

In [47]:
using Combinatorics

In [48]:
Set( collect(permutations(state_list[2])) )

Set{Vector{Int64}} with 2 elements:
  [2, 1]
  [1, 2]

In [49]:
basis_mb.shape

3

In [50]:
PMB = zeros(ComplexF64,basis_mb.shape,basis_mb.shape)
for iMB in 1:basis_mb.shape
    i_occ_lists = Set( collect(permutations(state_list[iMB])) )
    ilistlen = length(i_occ_lists)
    for jMB in 1:basis_mb.shape
        j_occ_lists = Set( collect(permutations(state_list[jMB])) )
        jlistlen = length(j_occ_lists)
        for (iocc_ind,iocc) in enumerate(i_occ_lists)
            for (jocc_ind,jocc) in enumerate(j_occ_lists)
                P1val=1
                for ind in 1:PN
                    P1val *= P1M[iocc[ind],jocc[ind]]
                    println(iMB,'\t',jMB,'\t', iocc_ind,jocc_ind, '\t', P1val)
                end    
                PMB[iMB,jMB] += P1val/sqrt(ilistlen*jlistlen) 
            end
        end
    end
end

1	1	11	-0.7071067811865475 + 0.0im
1	1	11	0.4999999999999999 - 0.0im
1	2	11	-0.7071067811865475 + 0.0im
1	2	11	0.4999999999999999 - 0.0im
1	2	12	-0.7071067811865475 + 0.0im
1	2	12	0.4999999999999999 - 0.0im
1	3	11	-0.7071067811865475 + 0.0im
1	3	11	0.4999999999999999 - 0.0im
2	1	11	-0.7071067811865475 + 0.0im
2	1	11	0.4999999999999999 - 0.0im
2	1	21	-0.7071067811865475 + 0.0im
2	1	21	0.4999999999999999 - 0.0im
2	2	11	0.7071067811865475 + 0.0im
2	2	11	-0.4999999999999999 + 0.0im
2	2	12	-0.7071067811865475 + 0.0im
2	2	12	0.4999999999999999 - 0.0im
2	2	21	-0.7071067811865475 + 0.0im
2	2	21	0.4999999999999999 - 0.0im
2	2	22	-0.7071067811865475 + 0.0im
2	2	22	-0.4999999999999999 + 0.0im
2	3	11	0.7071067811865475 + 0.0im
2	3	11	-0.4999999999999999 + 0.0im
2	3	21	-0.7071067811865475 + 0.0im
2	3	21	-0.4999999999999999 + 0.0im
3	1	11	-0.7071067811865475 + 0.0im
3	1	11	0.4999999999999999 - 0.0im
3	2	11	0.7071067811865475 + 0.0im
3	2	11	-0.4999999999999999 + 0.0im
3	2	12	-0.7071067811865475 + 0.0

In [51]:
P1M

2×2 Matrix{ComplexF64}:
 -0.707107+0.0im  -0.707107+0.0im
 -0.707107+0.0im   0.707107+0.0im

In [52]:
PMB

3×3 Matrix{ComplexF64}:
      0.5+0.0im   0.707107+0.0im        0.5+0.0im
 0.707107+0.0im        0.0+0.0im  -0.707107+0.0im
      0.5+0.0im  -0.707107+0.0im        0.5+0.0im

In [53]:
UU

3-element Vector{Ket{ManyBodyBasis{NLevelBasis{Int64}, QuantumOpticsBase.SortedVector{Vector{Int64}, Base.Order.ReverseOrdering{Base.Order.ForwardOrdering}}, UInt64}, Vector{ComplexF64}}}:
 Ket(dim=3)
  basis: ManyBody(onebodybasis=NLevel(N=2), states:3)
 0.4693716741659837 + 0.17230853574510274im
 0.6637917873992998 + 0.2436810681633735im
 0.4693716741659841 + 0.17230853574510263im
 Ket(dim=3)
  basis: ManyBody(onebodybasis=NLevel(N=2), states:3)
 -0.6404482189056551 + 0.2997099913259059im
            -2.8e-16 - 2.2e-16im
  0.6404482189056546 - 0.29970999132590576im
 Ket(dim=3)
  basis: ManyBody(onebodybasis=NLevel(N=2), states:3)
 -0.0223415745544634 + 0.4995006046507125im
 0.03159575773969178 - 0.7064005295105991im
 -0.0223415745544635 + 0.4995006046507125im

In [54]:
PMB*UU[3].data

3-element Vector{ComplexF64}:
 -4.85722573273506e-17 + 5.551115123125783e-17im
 7.979727989493313e-17 + 0.0im
 -0.044683149108926845 + 0.9990012093014248im

In [55]:
UUt

3-element Vector{Ket{ManyBodyBasis{SubspaceBasis{Vector{Int64}, NLevelBasis{Int64}, Ket{NLevelBasis{Int64}, Vector{ComplexF64}}, 0x868ccc38620d16b5, UInt64}, QuantumOpticsBase.SortedVector{Vector{Int64}, Base.Order.ReverseOrdering{Base.Order.ForwardOrdering}}, UInt64}, Vector{ComplexF64}}}:
 Ket(dim=3)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=2), states:2), states:3)
 1.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 Ket(dim=3)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=2), states:2), states:3)
 0.0 + 0.0im
 1.0 + 0.0im
 0.0 + 0.0im
 Ket(dim=3)
  basis: ManyBody(onebodybasis=Subspace(superbasis=NLevel(N=2), states:2), states:3)
 0.0 + 0.0im
 0.0 + 0.0im
 1.0 + 0.0im

In [56]:
PMB

3×3 Matrix{ComplexF64}:
      0.5+0.0im   0.707107+0.0im        0.5+0.0im
 0.707107+0.0im        0.0+0.0im  -0.707107+0.0im
      0.5+0.0im  -0.707107+0.0im        0.5+0.0im

In [62]:
for iMB in 1:basis_mb.shape
    println( norm( UU[iMB]'*Operator(basis_mb,basis_mb_tilde,PMB)*UUt[iMB] ))
end

0.9999999999999996
0.9999999999999999
0.9999999999999999


In [58]:
basis_mb.onebodybasis

NLevel(N=2)

In [59]:
basis_mb_tilde.onebodybasis.basisstates

2-element Vector{Ket{NLevelBasis{Int64}, Vector{ComplexF64}}}:
 Ket(dim=2)
  basis: NLevel(N=2)
 -0.7071067811865475 + 0.0im
 -0.7071067811865475 + 0.0im
 Ket(dim=2)
  basis: NLevel(N=2)
 -0.7071067811865475 + 0.0im
  0.7071067811865475 + 0.0im